In [1]:
import sqlite3
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import csv
import json
conn= sqlite3.connect('three_ships.db')
cursor= conn.cursor()

In [2]:
root_url= 'https://docs.google.com/spreadsheets/d/1wcv-CskyGrVali7OIDGy-hAf_vhknWFIOswWZOcnc3M/edit#gid=165658315'
sheet_name = 'results-20230206-135407'

In [3]:
def extract_id(root_url):
    match = re.search(r'/d/([-\w]+)/', root_url)
    if match:
        id = match.group(1)
    return id

In [4]:
def output_csv(root_url,sheet_name):
    SHEET_ID= extract_id(root_url)
    SHEET_NAME= sheet_name
    url=f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
    return pd.read_csv(url)

In [5]:
df=output_csv(root_url,sheet_name)
df

,results_position,results_title,results_link
0,1,"THE BEST 10 Movers in Independence, MO - Yelp",https://www.yelp.com/search?cflt=movers&find_l...
1,2,"The 21 Best Independence, MO Movers - HireAHelper",https://www.hireahelper.com/movers/independenc...
2,3,"The 10 Best Movers in Independence, MO (with F...",https://www.thumbtack.com/mo/independence/movers
3,4,"Top 10 Best Movers in Independence, MO - Angie...",https://www.angi.com/companylist/us/mo/indepen...
4,5,"Independence, MO Movers - All My Sons Moving &...","https://www.allmysons.com/independence,_mo_mov..."
...,...,...,...
93,94,The Movers: A Saga of the Scotch-Irish,https://books.google.com/books?id=xum1aqfq-asc...
94,95,Body Movers - Volume 1 - Page 96 - Google Book...,https://books.google.com/books?id=vgcpxazvsu4c...
95,96,Fast Movers - Page 9 - Google Books Result,https://books.google.com/books?id=q917ysh6szac...
96,97,Sit Back and Relax! Our Local Movers in Overla...,https://mikehammermoving.com/sit-back-local-mo...


In [6]:
def create_moving_companies_table(cursor, conn):
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS moving_companies  (
    results_position INTEGER PRIMARY KEY,
    results_title TEXT,
    results_link TEXT
    )
    """)
    conn.commit()

In [7]:
create_moving_companies_table(cursor, conn)

In [8]:
df.to_sql('moving_companies', con=conn, if_exists='replace')

98

In [9]:
def query_moving_companies(query, conn, pd=pd):
    result_df = pd.read_sql(query, conn)
    return result_df

In [10]:
query_1="""
SELECT *
FROM moving_companies
WHERE results_link LIKE '%Independence%' OR results_title LIKE '%Independence%'
"""
result_df_1 = query_moving_companies(query_1, conn)
result_df_1

,index,results_position,results_title,results_link
0,0,1,"THE BEST 10 Movers in Independence, MO - Yelp",https://www.yelp.com/search?cflt=movers&find_l...
1,1,2,"The 21 Best Independence, MO Movers - HireAHelper",https://www.hireahelper.com/movers/independenc...
2,2,3,"The 10 Best Movers in Independence, MO (with F...",https://www.thumbtack.com/mo/independence/movers
3,3,4,"Top 10 Best Movers in Independence, MO - Angie...",https://www.angi.com/companylist/us/mo/indepen...
4,4,5,"Independence, MO Movers - All My Sons Moving &...","https://www.allmysons.com/independence,_mo_mov..."
5,7,8,"Cheap Movers in Independence, MO - Great Guys ...",https://www.greatguysmovers.com/movers/missour...
6,8,9,"Movers Near Me in Independence, MO - Moving Help",https://www.movinghelp.com/movers/mo/independence
7,10,11,Movers Independence MO ( Looking for Moving Co...,https://afwatmovers.com/movers/independence-mo/
8,14,15,Best Movers in Independence MO - Get Free Movi...,https://www.movers.com/moving-companies/mo-mis...
9,15,16,"Independence Movers - Independence, MO Moving",https://www.danielsmoving.com/areas-served/ind...


In [11]:
query_2="""
SELECT *
FROM moving_companies
WHERE results_link LIKE '.com/movers/%/%'
"""
result_df_2 = query_moving_companies(query_2, conn)
result_df_2

,index,results_position,results_title,results_link


These are a few things I could look to at to inform my tagging:


    1. Logo:If a website showcases its own branding and logo predominantly, it is probably a local moving service provider. Conversely, if the website only features branding or mentions of other moving companies, it is likely an aggregator.
    2. Links to other websites: The presence of links on a website directing to other websites providing moving services could suggest that the website is serving as an aggregator. 
    3. Service offerings: It's probable that a local moving provider will advertise their own services, while an aggregator will exhibit services from several providers.
    3. URL: The structure of a URL could provide hints regarding the type of website. A URL that contains "movers/{state}/{city}" may suggest a local moving provider, while a URL containing "moving-companies" or "moving-services" might indicate an aggregator.
    4. Reviews:If the website shows only one company's reviews, it probably represents a local moving service provider. However, if the website presents reviews of various companies, it probably acts as an aggregator.
    5.Contact information:It's likely that a local moving provider will exhibit their own contact information, like an email address or phone number, whereas an aggregator may not do so.

In [12]:
def get_social_media_links(company_info):
    facebook = None
    instagram = None
    twitter = None
    for html in company_info:
        if 'facebook' in html:
            soup = BeautifulSoup(html, 'html.parser')
            my_tag = soup.find('a')
            href_value = my_tag.get('href')
            if href_value:
                facebook = href_value.split()[0]
        elif 'instagram' in html:
            soup = BeautifulSoup(html, 'html.parser')
            my_tag = soup.find('a')
            href_value = my_tag.get('href')
            if href_value:
                instagram = href_value.split()[0]
        elif 'twitter' in html:
            soup = BeautifulSoup(html, 'html.parser')
            my_tag = soup.find('a')
            href_value = my_tag.get('href')
            if href_value:
                twitter = href_value.split()[0]
    return facebook, instagram, twitter


In [13]:
# First I downloaded the json file from google drive to this directory and
# Load the data from the file
def load_json_file(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

In [14]:
def extract_social_media_links(json_file, csv_file):
    data=load_json_file(json_file)
    ## I created the list to append the results
    output = []
    for company_id, company_info in data.items():
        facebook, instagram, twitter = get_social_media_links(company_info)

        # Add the company's information to the output list
        output.append({
            'company_id': company_id,
            'facebook': facebook,
            'instagram': instagram,
            'twitter': twitter,
        })

    # Write the results to a CSV file
    with open(csv_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['company_id', 'facebook', 'instagram', 'twitter'])
        writer.writeheader()
        writer.writerows(output)

In [15]:
#Call the function to get the csv
extract_social_media_links('hvac-links.json', 'hvac_companies_social_media.csv')